In [69]:
import os 
import pandas as pd 
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms,models,utils
from tqdm.notebook import tqdm
from torch import nn
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils.tensorboard import SummaryWriter

In [70]:
train_path = "train"
test_path = "test"
data_root = "train"
csv_path = "result"
tensorboard_path = "../tensorboard"
model_save_path = "../model"

In [71]:
class MyDataset(Dataset):
    def __init__(self, parent_folder: str, train=True, transform=None):
        self.parent_folder = parent_folder
        self.data = []
        self.train_flag = train
        if transform is None:
            self.transform = transforms.Compose(
            [
                transforms.Resize(size = (224,224)),#尺寸规范
                transforms.Grayscale(num_output_channels=3),  # 将单通道图像转换为3通道
                transforms.ToTensor(),   #转化为tensor
                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
            ])
        else:
            self.transform = transform
        #
        subfolders = os.listdir(parent_folder)
        for subfolder in subfolders:
            folder_path = os.path.join(parent_folder, subfolder)
            if os.path.isdir(folder_path):
                file_list = os.listdir(folder_path)
                for file_name in file_list:
                    file_path = os.path.join(folder_path, file_name)
                    self.data.append((file_path, subfolder))  # 存储文件路径和对应的子文件夹名

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        img_path, subfolder = self.data[idx]
        img = Image.open(img_path)
        img = self.transform(img)

        return img, label, subfolder  # 返回图像和子文件夹名作为标签


In [72]:
class MyDataset(Dataset):
    def __init__(self, data_dir, train=True, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.file_list = []
        self.labels = []

        # Loop through the "Patients" and "Normal" folders
        for label, folder_name in enumerate(["Patients", "Normal"]):
            folder_path = os.path.join(data_dir, folder_name)
            if os.path.exists(folder_path):
                for filename in os.listdir(folder_path):
                    if filename.endswith(".tif"):
                        self.file_list.append(os.path.join(folder_path, filename))
                        self.labels.append(label)

    def __len__(self):
        return len(self.file_list)

    def __getitem__(self, idx):
        img_path = self.file_list[idx]
        label = self.labels[idx]
        img = Image.open(img_path)

        if self.transform:
            img = self.transform(img)

        return img, label

# Usage example:
# Define your data directory and transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])


In [73]:
train_ds = MyDataset(train_path)
test_ds = MyDataset(test_path,train=False)
for i, item in enumerate(train_ds):
    print(item)
    break

(<PIL.TiffImagePlugin.TiffImageFile image mode=L size=304x510 at 0x21F33E875B0>, 0)


In [74]:
train_loader = torch.utils.data.DataLoader(train_ds, batch_size=32,
                                            shuffle=True, pin_memory=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_ds, batch_size=32,
                                            shuffle=True, pin_memory=True, num_workers=0)

ValueError: num_samples should be a positive integer value, but got num_samples=0